In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier

from imblearn.over_sampling import SMOTE

# Veri yükleme
data = pd.read_excel('paper.xlsx')

# Veri genel bilgisi ve önizleme
print(data.info())
print(data.head())

# Eksik veri kontrolü
print("Eksik veriler:\n", data.isnull().sum())

# GPA'yı sınıflara ayırma
def gpa_to_label(gpa):
    if gpa >= 3.0:
        return 'High'
    elif gpa >= 2.0:
        return 'Medium'
    else:
        return 'Low'

data['GPA_Class'] = data['GPA'].apply(gpa_to_label)

# Eksik verileri temizleme
data = data.dropna()

# Sayısal sütunlar
numeric_columns = ['Time_Friends', 'Time_SocicalMedia', 'Time_Studying']
numeric_columns = [col for col in numeric_columns if col in data.columns]

# Sayısal sütunları ölçeklendirme
scaler = StandardScaler()
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Hedef değişken ve özellikler
y = data['GPA_Class']
X = data.drop(['GPA', 'GPA_Class'], axis=1)

# Korelasyon matrisi görselleştirme
numeric_data = data.select_dtypes(include=[np.number])
plt.figure(figsize=(10,8))
sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Korelasyon Matrisi")
plt.show()

# Orijinal sınıf dağılımı
print("Orijinal sınıf dağılımı:\n", y.value_counts())

# SMOTE ile oversampling
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
print("Oversampling sonrası sınıf dağılımı:\n", pd.Series(y_resampled).value_counts())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# === ROC için sınıfları binarize et ===
y_train_bin = label_binarize(y_train, classes=['Low', 'Medium', 'High'])
y_test_bin = label_binarize(y_test, classes=['Low', 'Medium', 'High'])

# KNN modelini One-vs-Rest ile eğit
model = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
model.fit(X_train, y_train_bin)

# Tahmin
y_score = model.predict_proba(X_test)

# ROC eğrisi çizimi
fpr, tpr, roc_auc = dict(), dict(), dict()
for i in range(y_test_bin.shape[1]):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# ROC çizimi
plt.figure(figsize=(8,6))
colors = ['blue', 'orange', 'green']
labels = ['Low', 'Medium', 'High']
for i in range(len(labels)):
    plt.plot(fpr[i], tpr[i], color=colors[i], label=f'{labels[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Eğrisi - KNN (k=5)')
plt.legend(loc='lower right')
plt.grid()
plt.tight_layout()
plt.show()

# Performans sonuçları
y_pred = model.predict(X_test)
y_pred_label = np.argmax(y_pred, axis=1)
y_test_label = np.argmax(y_test_bin, axis=1)
print("Doğruluk:", accuracy_score(y_test_label, y_pred_label))
print("Sınıflandırma Raporu:\n", classification_report(y_test_label, y_pred_label, target_names=['Low', 'Medium', 'High']))
print("Confusion Matrix:\n", confusion_matrix(y_test_label, y_pred_label))

# Boxplot ile sayısal değişkenlerin GPA sınıflarına göre dağılımı
for col in numeric_columns:
    plt.figure(figsize=(8,6))
    sns.boxplot(x='GPA_Class', y=col, data=data)
    plt.title(f"{col} Dağılımı GPA Sınıflarına Göre")
    plt.show()
